In [15]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [16]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=next&oldid=1013111853'
url

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find(id="firstHeading")
print(title.string)

List of postal codes of Canada: M


In [17]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(10)

,Borough,Neighborhood,Postal Code
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern, Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill, Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


# Table requested in the task is added here

# Shape as shown below

In [19]:
df.shape

(103, 3)

# CSV is loaded because google api is not working

In [20]:
df_coordinats = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinats.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Combination of CSV and Wikipedia Data

In [23]:
df_new = pd.merge(df,df_coordinats,on='Postal Code',how='left')
df_new.head(10)

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
5,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
7,North York,Don Mills North,M3B,43.745906,-79.352188
8,East York,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937
9,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937


# Clustering

In [25]:
from sklearn.cluster import KMeans

In [27]:
k_means = KMeans(init = "k-means++",n_clusters=5, n_init=12)
data = df_new[['Latitude','Longitude']].values

In [28]:
k_means.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=12, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [30]:
k_means.labels_

array([0, 0, 4, 2, 4, 1, 3, 0, 0, 4, 2, 1, 3, 0, 0, 4, 2, 1, 3, 0, 4, 2,
       3, 0, 4, 4, 3, 0, 2, 0, 4, 4, 3, 0, 2, 0, 4, 4, 3, 0, 2, 4, 4, 4,
       0, 0, 2, 0, 4, 2, 1, 3, 2, 2, 4, 2, 2, 1, 0, 2, 2, 2, 2, 1, 1, 3,
       2, 4, 4, 2, 1, 0, 2, 2, 4, 4, 1, 1, 3, 4, 4, 1, 0, 4, 4, 3, 4, 4,
       1, 1, 0, 4, 4, 1, 1, 3, 4, 4, 1, 4, 4, 1, 1])

# Clustering output is added as "Group"

In [31]:
df_new.insert(1,'Group',k_means.labels_)

In [33]:
df_new.head(10)

,Borough,Group,Neighborhood,Postal Code,Latitude,Longitude
0,North York,0,Parkwoods,M3A,43.753259,-79.329656
1,North York,0,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,4,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,2,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,4,Ontario Provincial Government,M7A,43.662301,-79.389494
5,Etobicoke,1,Islington Avenue,M9A,43.667856,-79.532242
6,Scarborough,3,"Malvern, Rouge",M1B,43.806686,-79.194353
7,North York,0,Don Mills North,M3B,43.745906,-79.352188
8,East York,0,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937
9,Downtown Toronto,4,"Garden District, Ryerson",M5B,43.657162,-79.378937


In [35]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# Average Lat and Long are used for setting of MAP

In [38]:
latitude = df_new['Latitude'].mean()
longitude = df_new['Longitude'].mean()

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
kclusters=5

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_new['Latitude'], df_new['Longitude'], df_new['Neighborhood'], df_new['Group']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters